# create a printer
This module creates a printer device and link it to the fixture.

Module interaction:

    Environement selection
    Premise selection
    Floor selection
    Area selection
    Zone selection
    Fixture selection
    => printer creation
            ID : identifier of the device (unsigned integer), must be unique in the organization. Auto generated by default
            IP (mandatory) : ip address of the device on the local network, must be unique in the premise (unique in the organization for ShopCX)
            NAME (mandatory) : friendly name of the device
            MAC : MAC address of the device

## Select an environment

In [ ]:
%load_ext autoreload
%autoreload 2
from libConfUtils import chooseEnv as chooseEnv
from libConfUtils import deviceUtils as deviceUtils
from libConfUtils import povUtils as povUtils
from libConfUtils import uiUtils as uiUtils
from libConfUtils import premiseNhubUtils as premiseNhubUtils
from libConfUtils import settingsUtils as settingsUtils

import ipaddress

In [ ]:
#%%
status = True
myEnv = chooseEnv.choosePremise()
if not myEnv:
    status = False

In [ ]:
#%% display choice
if status :
    print("Organization:" + myEnv.get('urlDP') + " Premise code "+  myEnv.get('PremiseCode'))

In [ ]:
myEnv = chooseEnv.chooseArea(myEnv)
myEnv = chooseEnv.chooseZone(myEnv)
myEnv = chooseEnv.chooseFixture(myEnv, dialogTitle='Select fixture for printer ')
printerFixture = myEnv['FixtureId']
if printerFixture:
    print('fixture for the printer: ',printerFixture)
else:
    text = 'Printer Fixture is a mandatory'
    uiUtils.displayMessage('Printer creation',text)
    print(text)
    status = False

Open form to enter printer parameters:

* IP : ip address of the printer on the lcoal network, must be unique in the premise (unique in the organization for ShopCX)
* ID : identifier of the printer (unsigned integer), must be unique in the organization
* NAME : friendly name of the printer (will be displayed in "printing dashboard")
* MAC : MAC address of the printer, can be fake in the organization
* FIXTURE : fixture SGLN (ex: 3663328.00001.1) that should be linked to the printer

In [ ]:
#%% 
if status:
    myForm = uiUtils.inputWithForm('printer definition',['IP','NAME','MAC'])

    printerIp = myForm.get('IP')
    #printerId = myForm.get('ID(integer)') # NUMBER
    printerName = myForm.get('NAME')
    printerMac = myForm.get('MAC') # can be fake
    #printerFixture = myForm.get('FIXTURE')

    # check entries
#    if printerId and not printerId.isdigit():

#    if not printerId :
#        text = 'Printer Id is a mandatory field '
#        uiUtils.displayMessage('Printer creation',text)
#        print(text)
#        status = False
#    elif not printerId.isdigit():
#        text = 'printer Id is not a number '
#        uiUtils.displayMessage('Printer creation',text)
#        print(text)
#        status = False
        
    if not printerName:
        text = 'Printer Name is a mandatory field '
        uiUtils.displayMessage('Printer creation',text)
        print(text)
        status = False
        
    try:    
        tmp = ipaddress.ip_address(printerIp)
    except:
        text = 'Printer IP address is not correct '
        uiUtils.displayMessage('Printer creation',text)
        print(text)
        status = False   


Use REST API to create the printer

In [ ]:
#%% CHECK ID UNICITY
checkIpUnicity = True

In [ ]:
#%% CHECK IP UNICITY in the Premise
# DEV NOTE : should be integrated in the backEnd
if status and checkIpUnicity:
    res = deviceUtils.checkIPUnicityOnThePremise(myEnv,printerIp)
    if not res.get('status'):
        text = res.get('message')
        text  = text + '\n'+ ' Printer can not been created'
        uiUtils.displayMessage('Printer create',text)
        status = False

In [ ]:
#%%
if status:
    printerDefinition = deviceUtils.createPrinter(myEnv,printerIp,printerName,printerMac,printerFixture)
    if not printerDefinition:
        text = 'Printer not created correctly '
        uiUtils.displayMessage('Printer creation',text)
        print(text)
        status = False  
    else:
        printerIdReturn = printerDefinition.get('id')
        if not printerIdReturn:
            text = 'Printer not created correctly, can t retreive the ID '
            uiUtils.displayMessage('Printer creation',text)
            print(text)
            status = False
        else:
            printerId = printerIdReturn


Check that the printer is correcty created

In [ ]:
if status:
    deviceDefinition = deviceUtils.getPrinter(myEnv,printerId)
    if not deviceDefinition:
        text = 'Printer not created correctly '
        uiUtils.displayMessage('Printer creation',text)
        print(text)
        status = False          

display information

In [ ]:
#%%
if status:

    text = "Premise: " + str(myEnv.get('PremiseCode'))
    text  = text +'\n' + 'Printer ID: ' +  str(printerId)
    text  = text +'\n' + 'Printer IP: ' +  str(deviceDefinition.get('uri'))
    text  = text +'\n' + 'Printer MAC: ' +  str(deviceDefinition.get('mac'))
    text  = text +'\n' + 'Printer Fixture: ' +  str(deviceDefinition.get('fixtureId'))


    uiUtils.displayMessage('Printer parameters',text)